In [80]:
import pandas as pd
import numpy as np
import nltk
import networkx as nx
from rouge import Rouge
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import re
import nltk.data
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import string

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
rouge_=Rouge()
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /home/sunild/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sunild/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [81]:
"""
FIXED VARIABLES DECLARED
"""
EMBEDDING_SIZE=100 # OPTIONS: 50,100,200,300
SENT_SUMMARY_COUNT=5
SENT_SUMMARY_THERSHOLD=0.35
HI_DATA_PATH='../data/hi-data/test.csv' # BOTH ARTICLES AND SUMMARIES
EN_DATA_PATH_SRC='../data/preprocessed_truncated/test.txt.src.tokenized.fixed.cleaned.final.truncated.txt' # ONLY ARTICLES
EN_DATA_PATH_TGT='../data/preprocessed_truncated/test.txt.tgt.tokenized.fixed.cleaned.final.truncated.txt' # ONLY SUMMARIES
HI_EMBEDDINGS_PATH='../data/glove/hi/hi-d100-glove.txt'
EN_EMBEDDINGS_PATH='../data/glove/en/glove.6B.100d.txt'

In [82]:
"""
HELPER FUNCTIONS FOR MAIN SUMMARY GENERATION FUNC
"""
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


def load_glove_model(file_path):
    """
    function to load glove embeddings from txt file into a dict
    """
    glove_model = {}
    with open(file_path,'r',encoding="utf-8",errors='ignore') as f:
        try:
            for i,line in enumerate(f):
                # if i%10000==0:
                #     print(i)
                split_line = line.split()
                word = split_line[0]
                embedding = np.array(split_line[-EMBEDDING_SIZE:], dtype=np.float64)
                glove_model[word] = embedding
        except Exception as e:
            print(e)
            print(line,i)
    print(f"total {len(glove_model)} words loaded from glove model, each of size {EMBEDDING_SIZE}.")
    return glove_model

# load glove embeddings for both hindi and english
glove_embeddings=load_glove_model(file_path=EN_EMBEDDINGS_PATH)
# hi_glove_embeddings=load_glove_model(file_path=HI_EMBEDDINGS_PATH)
# save all glove embeddings in a single dict
# this takes care of both languages that are present in hindi texts; although it makes overall process a bit slow
# glove_embeddings.update(hi_glove_embeddings)    

total 400000 words loaded from glove model, each of size 100.


In [83]:
stop_words = set(stopwords.words('english'))
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

<>:2: DeprecationWarning: invalid escape sequence \,


In [84]:
documents=["gop eyes gains as voters in 11 states pick governors enlarge this image toggle caption jim cole/ap jim cole/ap voters in 11 states will pick their governors tonight , and republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two-thirds of the nation ' s top state offices . eight of the gubernatorial seats up for grabs are now held by democrats ; three are in republican hands . republicans currently hold 29 governorships , democrats have 20 , and rhode island ' s gov . lincoln chafee is an independent . polls and race analysts suggest that only three of tonight ' s contests are considered competitive , all in states where incumbent democratic governors aren ' t running again : montana , new hampshire and washington . while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and indiana . democrats are likely to hold on to their seats in west virginia and missouri , and are expected to notch safe wins in races for seats they hold in vermont and delaware . holding sway on health care while the occupant of the governor ' s office is historically far less important than the party that controls the state legislature , top state officials in coming years are expected to story_separator_special_tag gop eyes gains as voters in 11 states pick governors jim cole / ap i jim cole / ap voters in 11 states will pick their governors tonight , and republicans appear on track to increase their numbers by at least one , and with the potential to extend their hold to more than two-thirds of the nation ' s top state offices . eight of the gubernatorial seats up for grabs today are now held by democrats ; three are in republican hands . republicans currently hold 29 governorships , democrats have 20 ; and rhode island ' s gov . lincoln chafee is an independent . polls and race analysts suggest that only three of tonight ' s contests are considered competitive , all in states where incumbent democratic governors aren ' t running again : montana , new hampshire and washington . while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and indiana . democrats are likely hold on to their seats in west virginia and missouri ; and expected to notch safe wins in races for seats they hold in vermont and delaware . holding sway on health care while the occupant of the governor ' s office is historically far less important than the party that controls the state legislature , top state officials in coming years are expected to"]

In [85]:
def get_sent_vect(probe_text):
    """
    func to return sent vectors
    """

    break_sentences=tokenizer.tokenize(probe_text)
    clean_sentences=[]
    for item in break_sentences:
      item=item.translate(str.maketrans('', '', string.punctuation))
      item=' '.join( [w for w in item.split() if len(w)>1] )
      words = word_tokenize(item)
      words = [word for word in words if len(words)>1]
      words = [word.lower() for word in words]
      words = [word for word in words if word not in stop_words]
      item=' '.join(words)    
      clean_sentences.append(item)
    cln_summ=''
    cln_summ = '.'.join(clean_sentences)
    # print(cln_summ)
    #print(cln_summ)
    tfidf = CountVectorizer()
    response = tfidf.fit_transform([cln_summ])
    #print(response)
    unique_word_list = tfidf.get_feature_names_out()
    tf_idf={}
    for col in response.nonzero()[1]:
      tf_idf[unique_word_list[col]]= response[0, col]
    svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
    for i in range(0,len(clean_sentences)):
      sen=clean_sentences[i].split()
      for val in sen:
        #print(val)
        index=np.where(unique_word_list == val)
        index_length=len(index[0])
        if(index_length == 1):
          svd_matrix[index,i]=tf_idf[val]
    #print(svd_matrix)
    u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
    sum_sigma=np.sum(sigma)
    iter_sum=0
    check_coverage=0
    for i in range(0,len(sigma)):
      iter_sum=iter_sum+sigma[i]
      percent_contribution=iter_sum/sum_sigma
      #print("the percent contributuin is ",percent_contribution)
      if(percent_contribution<0.70):
        check_coverage=i+1
    u_trimmed=u[:,:check_coverage]
    v_trimmed=v_trans[:check_coverage,:]

    sent_vects=[]
    for i,sent in enumerate(clean_sentences):
        # sent_words=sent.split(' ')
        tmp_=0
        for k,concept_weight in enumerate(v_trimmed[:,i]):
            foo_=0
            for j,word in enumerate(unique_word_list):
                if word in glove_embeddings:
                    # print('it is')
                    foo_+=u_trimmed[j,k]*glove_embeddings[word]
            tmp_+=concept_weight*foo_
        
        sent_vects.append(tmp_)
      
    return sent_vects

In [86]:
def generate_summary(article_text):
    """
    function to get summary of article text
    """
    sent_tokens=sent_tokenize(article_text)
    sent_vects=get_sent_vect(article_text)
    
    total_sents=len(sent_vects)
    similarity_mat=np.zeros((total_sents,total_sents))
    
    for i in range(total_sents):
        for j in range(total_sents):
            # this way diag entries will be zeros
            if i!=j:
                similarity_mat[i][j]=cosine_similarity(sent_vects[i],sent_vects[j])

    # create graph from similarity matrix
    network_graph=nx.from_numpy_array(similarity_mat)
    """
    apply pagerank algo to get scores
    """
    scores=nx.pagerank(network_graph, alpha=0.85, max_iter=1000)
    sorted_scores = sorted(((scores[i],sent_token) for i,sent_token in enumerate(sent_tokens)), reverse=True)

    # TODO: implement thershold version
    # in case some article does not have enough sentences
    if total_sents > SENT_SUMMARY_COUNT:
        summary_text=' '.join([sorted_scores[i][1] for i in range(SENT_SUMMARY_COUNT)])
    else:
        summary_text=' '.join([sorted_scores[i][1] for i in range(total_sents)])

    return summary_text


In [87]:
generate_summary(documents[0])

"while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and indiana . while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and indiana . democrats are likely to hold on to their seats in west virginia and missouri , and are expected to notch safe wins in races for seats they hold in vermont and delaware . democrats are likely hold on to their seats in west virginia and missouri ; and expected to notch safe wins in races for seats they hold in vermont and delaware . polls and race analysts suggest that only three of tonight ' s contests are considered competitive , all in states where incumbent democratic governors aren ' t running again : montana , new hampshire and washington ."

In [88]:

for i in range(0,1):
  #print(i)
    probe_text=documents[i]
    ground_truth=clean_summ[i]
    break_sentences=tokenizer.tokenize(probe_text)
    clean_sentences=[]
    for item in break_sentences:
      item=item.translate(str.maketrans('', '', string.punctuation))
      item=' '.join( [w for w in item.split() if len(w)>1] )
      words = word_tokenize(item)
      words = [word for word in words if len(words)>1]
      words = [word.lower() for word in words]
      words = [word for word in words if word not in stop_words]
      item=' '.join(words)    
      clean_sentences.append(item)
    cln_summ=''
    cln_summ = '.'.join(clean_sentences)
    print(cln_summ)
    #print(cln_summ)
    tfidf = CountVectorizer()
    response = tfidf.fit_transform([cln_summ])
    #print(response)
    unique_word_list = tfidf.get_feature_names_out()
    tf_idf={}
    for col in response.nonzero()[1]:
      tf_idf[unique_word_list[col]]= response[0, col]
    svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
    for i in range(0,len(clean_sentences)):
      sen=clean_sentences[i].split()
      for val in sen:
        #print(val)
        index=np.where(unique_word_list == val)
        index_length=len(index[0])
        if(index_length == 1):
          svd_matrix[index,i]=tf_idf[val]
    #print(svd_matrix)
    u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
    sum_sigma=np.sum(sigma)
    iter_sum=0
    check_coverage=0
    for i in range(0,len(sigma)):
      iter_sum=iter_sum+sigma[i]
      percent_contribution=iter_sum/sum_sigma
      #print("the percent contributuin is ",percent_contribution)
      if(percent_contribution<0.70):
        check_coverage=i+1
    u_trimmed=u[:,:check_coverage]
    v_trimmed=v_trans[:check_coverage,:]
    # print("the shape of u_trimed is",u_trimmed.shape)
    # print("the shape of v trimmed is ",v_trimmed.shape)
    # print((unique_word_list))
    # chosen_sen_index=[]

gop eyes gains voters 11 states pick governors enlarge image toggle caption jim coleap jim coleap voters 11 states pick governors tonight republicans appear track increase numbers least one potential extend hold twothirds nation top state offices.eight gubernatorial seats grabs held democrats three republican hands.republicans currently hold 29 governorships democrats 20 rhode island gov.lincoln chafee independent.polls race analysts suggest three tonight contests considered competitive states incumbent democratic governors running montana new hampshire washington.state races remain close call republicans expected wrest north carolina governorship democratic control easily win gopheld seats utah north dakota indiana.democrats likely hold seats west virginia missouri expected notch safe wins races seats hold vermont delaware.holding sway health care occupant governor office historically far less important party controls state legislature top state officials coming years expected storyse

In [89]:
sent_vects=[]
for i,sent in enumerate(clean_sentences):
    # sent_words=sent.split(' ')
    tmp_=0
    for k,concept_weight in enumerate(v_trimmed[:,i]):
        foo_=0
        for j,word in enumerate(unique_word_list):
            if word in glove_embeddings:
                # print('it is')
                foo_+=u_trimmed[j,k]*glove_embeddings[word]
        tmp_+=concept_weight*foo_
    
    sent_vects.append(tmp_)


In [90]:
cosine_similarity(sent_vects[0],sent_vects[1])

0.7251169916658042